## Segementation analysis

In [1]:
import sys, os
cwd = os.getcwd()
print(cwd)
sys.path.insert(0, os.path.join(cwd, '..'))

/home/leonhard/Projects/AtlasOfSmoothSpaces/GestureLab/MpHandsRtclear/notebooks


In [3]:
from sklearn.mixture import GaussianMixture
import scipy
from segmentation import segmentation as sg 
from parseannotations import parseannotations1 as pa 

In [ ]:
fileName = "MariaMovementSequence_xyz_27Sept"
Annotations = {
    "hip_joint_right": [
        {"from": "00:05:12", "till": "00:07:11"},
        {"from": "00:44:03", "till": "00:46:03"}],
    "elbow_hinge_left": [
        {"from": "00:07:14", "till":"00:11:07"},
        {"from": "00:46:03", "till":"00:48:06"}],
    "pivot_right":[{"from": "00:48:07", "till": "00:49:13"}],
    "pivot_left":[{"from": "00:50:04", "till": "00:52:03"}],
    "rotation_shoulder_left": [{"from": "00:11:08" , "till":"00:14:02"}],
    "thumb_movement_left": [{"from": "00:14:03", "till": "00:18:10"}],
    "knee_hinge_right":[{"from":"00:18:11", "till": "00:21:01"}]
}

In [ ]:
## Load dataframe from file
df = sg.loadData(fileName=fileName, fps=15, fromRoot=False)
print('--> Dataframe has been loaded into df object.')
## rescale entries
df_rescaled = sg.rescaleDf(df)
print('--> Landmarks have been recentered around the nose and rescaled by the distance between the eyes.')
# ## get datapoints from annotations
result = sg.generateDataFromAnnotation(
    df=df_rescaled,
    anno=Annotations,
    batch_size=3, 
    timeToFrame=sg.kdenLiveTimeToFrame,
    asDict=True)
print('--> Data has been generated from annotations.')
resultAsArray = sg.turnDictDataIntoArray(result)
print('--> Data has been turned into one large array.')

In [ ]:
gmm = GaussianMixture(n_components=len(Annotations.keys()), random_state=0).fit(resultAsArray)

In [ ]:
df_rescaled.shape

In [ ]:
import numpy as np

In [ ]:
startIndex = 0
endIndex = df_rescaled.shape[0] - 4
goodindices = list()
for i in range(startIndex, endIndex):
    print(i)
    rg = range(startIndex,startIndex + 3)
    onlyCoordinateColumns = [c for c in df_rescaled.columns if c[0] in ['x', 'y', 'z']]
    three_consecutive_snaps = df_rescaled.loc[rg, onlyCoordinateColumns]
    new_input = np.array([three_consecutive_snaps.to_numpy().flatten()])
    probs = gmm.predict_proba(new_input)
    print(probs[0][-1])
    # # print(probs[0])
    # if (probs[0][-1]!=1.0):
    #     goodindices.append(i)
    #     print(probs)
    # print('at time i= {t} the prob is {prob}'.format(t=i,prob=str(probs)))
# three_consecutive_snaps.columns

In [ ]:
probs = gmm.predict_proba(resultAsArray[30:100])
probs

In [ ]:
probs = gmm.predict_proba(resultAsArray)
entropies = [scipy.stats.entropy(p) for p in probs]
max(entropies)

In [ ]:
import numpy as np
probs = gmm.predict_proba(np.array([resultAsArray[0]]))
entropies = [scipy.stats.entropy(p) for p in probs]
# max(entropies)

## Annotations from PieceMaker

In [22]:
r = pa.getAnnotationsFromMotionbankData('../data/hallein_werkschau_22h_combined-Oct24.csv')

In [5]:
dictData = pa.getData('../data/hallein_werkschau_22h_combined-Oct24.csv')

In [2]:
import os
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import pafy
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

def Euclidean(x,y):
    return np.sqrt((y[0]-x[0])**2 + (y[1] - x[1])**2)

In [3]:
# For webcam input:
from ast import ListComp

calcThis = True
findPose = True
fromWebCam = False

if calcThis:
  datadirectory = 'data'
  PoseData = list()
  # cap = cv2.VideoCapture(0)
  url   = "https://www.youtube.com/watch?v=6zQkbTzN8yY"
  video = pafy.new(url)
  best  = video.getbest()
  #documentation: https://pypi.org/project/pafy/

  cap = cv2.VideoCapture(best.url)
  # cap = cv2.VideoCapture(os.path.join('..', datadirectory, 'MariaDancingSequenceAnotherOne.webm'))
  i = 0
  imageName = "MariaMovementSequenceAnother_xyz_24Okt"
  # MaxRecordings = 10
  pose = mp_pose.Pose(
      min_detection_confidence=0.5,
      min_tracking_confidence=0.5)  # as pose:

  while cap.isOpened():
    
    i += 1
    
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      break

    try: # findPose:
    # Draw the pose annotation on the image.
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
      results = pose.process(image)
      mp_drawing.draw_landmarks(
          image,
          results.pose_landmarks,
          mp_pose.POSE_CONNECTIONS,
          landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
      
      newData = {coord + str(j):lm.__getattribute__(coord)  for j, lm in enumerate(results.pose_landmarks.landmark) for coord in ["x", "y", "z"]}
      PoseData.append(newData)
    
    except Exception as ex:
      print('didnt work', ex)
      # print(ex)

    # Flip the image horizontally for a selfie-view display.
    if fromWebCam:
      image = cv2.flip(image, 1)
    cv2.imshow('MediaPipe Pose', image)
    if (cv2.waitKey(5) & 0xFF == 27): ## or i>MaxRecording s :
      break
  cap.release()
  df = pd.DataFrame(PoseData)
  df.to_csv(os.path.join("..","data", "csv", imageName + ".csv"))
  cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object has no attribute 'landmark'
didnt work 'NoneType' object ha